In [1]:
from transformers import pipeline
import kaggle
import zipfile
import glob
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import DistilBertTokenizer

In [ ]:
os.getcwd()

'c:\\Users\\rune7\\Documents\\GitHub\\MLOps-Project-Detox\\notebooks'

In [ ]:
#unzip the data
with zipfile.ZipFile('../data/jigsaw-toxic-comment-classification-challenge.zip', 'r') as zip_ref:
    zip_ref.extractall('../data')

In [ ]:
files = glob.glob('../data/*.zip')
for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('../data')

In [2]:
train = pd.read_csv('../data/train.csv')

classes = ['toxic', 'severe_toxic', 'obsence', 'threat', 'insult', 'identity_hate']

#make a new column that contains vectors 

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model_inputs = tokenizer(train['comment_text'].tolist(), padding="max_length", truncation=True, return_tensors='pt')

KeyboardInterrupt: 

In [10]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [4]:
#we make evaluation dataset from test.csv
test = pd.read_csv('../data/test.csv')
test_labels = pd.read_csv('../data/test_labels.csv')
#replace -1 with 0 in test labels
test_labels = test_labels.replace(-1, 0)
test_labels.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000
mean,0.039761,0.002396,0.024098,0.001378,0.022375,0.004649
std,0.195399,0.048892,0.153355,0.037091,0.147899,0.068022
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
#convert test_labels to have
test_labels

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0,0,0,0,0,0
1,0000247867823ef7,0,0,0,0,0,0
2,00013b17ad220c46,0,0,0,0,0,0
3,00017563c3f7919a,0,0,0,0,0,0
4,00017695ad8997eb,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0,0,0,0,0,0
153160,fffd7a9a6eb32c16,0,0,0,0,0,0
153161,fffda9e8d6fafa9e,0,0,0,0,0,0
153162,fffe8f1340a79fc2,0,0,0,0,0,0


In [5]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from sklearn.model_selection import train_test_split
#import classlabel
from datasets.features import ClassLabel

y = np.array(train.drop(['id', 'comment_text'], axis=1))
X = np.array(train['comment_text'])


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#convert y_train and y_val to ClassLabel
y_train = y_train.astype('int')
y_val = y_val.astype('int')

X_test = np.array(test['comment_text'])
y_test = np.array(test_labels.drop(['id'], axis=1))



d = {'train':Dataset.from_dict({'label':y_train,'text':X_train}),
     'val':Dataset.from_dict({'label':y_val,'text':X_val}),
     'test':Dataset.from_dict({'label':y_test,'text':X_test})
     }

raw_dataset = DatasetDict(d)

In [6]:
#We convert dataset to tokenized pytorch tensors
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

  0%|          | 0/128 [00:00<?, ?ba/s]

  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/154 [00:00<?, ?ba/s]

In [40]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 127656
    })
    val: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 31915
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 153164
    })
})

In [46]:
tokenized_dataset['train'][0].keys()

dict_keys(['label', 'text', 'input_ids', 'attention_mask'])

In [7]:
#convert to torch dataset
from torch.utils.data import Dataset, DataLoader
class ToxicDataset(Dataset):
    def __init__(self, dataset):
        self.df = dataset
        self.label = self.df['label']
        self.text = self.df['text']
        self.input_ids = self.df['input_ids']
        self.attention_mask = self.df['attention_mask']
  
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        #convert to torch tensor
        input_ids = torch.tensor(self.input_ids[idx])
        attention_mask = torch.tensor(self.attention_mask[idx])
        label = torch.tensor(self.label[idx])
        return input_ids, attention_mask, label

In [8]:
train_dataset = ToxicDataset(tokenized_dataset['train'])
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [9]:
from transformers import AutoModelForSequenceClassification
checkpoint = "bert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [55]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

loading file vocab.txt from cache at C:\Users\rune7/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\rune7/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\tokenizer_config.json


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

loading configuration file config.json from cache at C:\Users\rune7/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



In [65]:
import gc

gc.collect()

torch.cuda.empty_cache()

!nvidia-smi

Fri Jan  6 16:56:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.37       Driver Version: 527.37       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   29C    P8     9W /  N/A |   1118MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
#train the model
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments("test-trainer", per_device_train_batch_size=96, per_device_eval_batch_size=96)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 127656
  Num Epochs = 3
  Instantaneous batch size per device = 96
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 3990
  Number of trainable parameters = 109486854
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/3990 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 8.00 GiB total capacity; 6.74 GiB already allocated; 0 bytes free; 6.83 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

2023-01-06 17:02:07,899 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F5B4C438B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /api/5288891/envelope/
2023-01-06 17:02:19,432 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F5B4C43B80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /api/5288891/envelope/
2023-01-06 18:34:51,859 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F5B4CE42E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /api/5288891/envelope/
